In [1]:
import tensorwrap as tf
from tensorwrap import torch
tf.__version__, torch.__version__

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


('0.0.0.3', '0.0.0.2')

In [2]:
x_train = torch.arange(1, 1000, 2)
y_train = x_train + 10

In [3]:
y_train

Array([  11,   13,   15,   17,   19,   21,   23,   25,   27,   29,   31,
         33,   35,   37,   39,   41,   43,   45,   47,   49,   51,   53,
         55,   57,   59,   61,   63,   65,   67,   69,   71,   73,   75,
         77,   79,   81,   83,   85,   87,   89,   91,   93,   95,   97,
         99,  101,  103,  105,  107,  109,  111,  113,  115,  117,  119,
        121,  123,  125,  127,  129,  131,  133,  135,  137,  139,  141,
        143,  145,  147,  149,  151,  153,  155,  157,  159,  161,  163,
        165,  167,  169,  171,  173,  175,  177,  179,  181,  183,  185,
        187,  189,  191,  193,  195,  197,  199,  201,  203,  205,  207,
        209,  211,  213,  215,  217,  219,  221,  223,  225,  227,  229,
        231,  233,  235,  237,  239,  241,  243,  245,  247,  249,  251,
        253,  255,  257,  259,  261,  263,  265,  267,  269,  271,  273,
        275,  277,  279,  281,  283,  285,  287,  289,  291,  293,  295,
        297,  299,  301,  303,  305,  307,  309,  3

In [4]:
# Expanding the axis:
x_train = torch.reshape(x_train, [-1, 1])

In [5]:
x_train

Array([[  1],
       [  3],
       [  5],
       [  7],
       [  9],
       [ 11],
       [ 13],
       [ 15],
       [ 17],
       [ 19],
       [ 21],
       [ 23],
       [ 25],
       [ 27],
       [ 29],
       [ 31],
       [ 33],
       [ 35],
       [ 37],
       [ 39],
       [ 41],
       [ 43],
       [ 45],
       [ 47],
       [ 49],
       [ 51],
       [ 53],
       [ 55],
       [ 57],
       [ 59],
       [ 61],
       [ 63],
       [ 65],
       [ 67],
       [ 69],
       [ 71],
       [ 73],
       [ 75],
       [ 77],
       [ 79],
       [ 81],
       [ 83],
       [ 85],
       [ 87],
       [ 89],
       [ 91],
       [ 93],
       [ 95],
       [ 97],
       [ 99],
       [101],
       [103],
       [105],
       [107],
       [109],
       [111],
       [113],
       [115],
       [117],
       [119],
       [121],
       [123],
       [125],
       [127],
       [129],
       [131],
       [133],
       [135],
       [137],
       [139],
       [141],
      

In [6]:
layer = torch.nn.layers.Linear(units = 1, input_shape = [1])

In [8]:
model = torch.nn.Sequential([
    layer
])

In [9]:
# Compiling the model:
model.compile(
    loss = tf.keras.losses.MSE,
    optimizer = tf.keras.optimizers.SGD(learning_rate = 0.1),
    accuracy = tf.keras.losses.MAE
)

In [10]:
model.fit(x_train = x_train, y_train = y_train, epochs = 100)

TypeError: grad requires real- or complex-valued inputs (input dtype that is a sub-dtype of np.inexact), but got int32. If you want to use Boolean- or integer-valued inputs, use vjp or set allow_int to True.

In [ ]:
model.__call__(tf.Variable([99]))

Array(1890.0881, dtype=float32)

In [ ]:
tf.Variable([1, 2, 3, 4, 5])

Array([1, 2, 3, 4, 5], dtype=int32)

In [ ]:
tf.jax.Array

jax.Array